### CONVOLUTIONAL NEURAL NETWORK WITH CUSTOM DATA

In [8]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
print ("Packages loaded")

Packages loaded


### LOAD DATA

In [9]:
# Load them!
# cwd = os.getcwd()
# loadpath = "../data/custom_data_flowers_64_64_rgb.npz"
# loadpath = "../data/data4vgg.npz"
# loadpath = "../data/custom_data_flowers_224_224_gray.npz"
loadpath = "../../DB/data/custom_data_flowers_256_256_gray.npz"

l = np.load(loadpath)

cnn_switch = 'basic'
# cnn_switch = 'vgg16'

# See what's in here
print (l.files)
for i in l.files:
    print(i + " \t: ", l[i].shape)


# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
use_gray = l['use_gray']
categories = l['categories']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]


if use_gray: 
    nimgch = 1
else:
    nimgch = 3

print ("TRAIN IMAGES : %d" % (ntrain))
print ("TEST IMAGES  : %d" % (ntest))
print ("IMAGE Channel: %d" % (nimgch))
print ("INPUT Dim.   : %d" % (dim))
print ("Image Size   :", imgsize)
print ("CLASSES      : %d" % (nclass))
print ("Categories   :", categories)

['trainimg', 'trainlabel', 'testimg', 'testlabel', 'imgsize', 'use_gray', 'categories']
trainimg 	:  (2936, 65536)
trainlabel 	:  (2936, 5)
testimg 	:  (734, 65536)
testlabel 	:  (734, 5)
imgsize 	:  (2,)
use_gray 	:  ()
categories 	:  (5,)
TRAIN IMAGES : 2936
TEST IMAGES  : 734
IMAGE Channel: 1
INPUT Dim.   : 65536
Image Size   : [256 256]
CLASSES      : 5
Categories   : ['daisy' 'dandelion' 'roses' 'sunflowers' 'tulips']


### DEFINE NETWORK

In [10]:
tf.set_random_seed(0)
n_input  = dim
n_output = nclass
if use_gray:
    weights = {'wc1': tf.Variable(tf.random_normal([5, 5, 1, 128], stddev=0.1))}
else:
    weights = {'wc1': tf.Varaible(tf.random_normal([5, 5, 3, 128], stddev=0.1))}

weights.update(
    {'wc2': tf.Variable(tf.random_normal([5, 5, 128, 128], stddev=0.1)),
     'wd1': tf.Variable(tf.random_normal([(int)(imgsize[0]/4*imgsize[1]/4)*128, 128], stddev=0.1)),
     'wd2': tf.Variable(tf.random_normal([128, n_output], stddev=0.1))
    }
)

biases   = {
    'bc1': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bc2': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1))
}

In [11]:
def conv_basic(_input, _w, _b, _keepratio, _use_gray):
    # INPUT
    if _use_gray:
        _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 1])
    else:
        _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 3])
    # CONVOLUTION LAYER 1
    _conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_input_r
        , _w['wc1'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
    # CONVOLUTION LAYER 2
    _conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_pool_dr1
        , _w['wc2'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
    # VECTORIZE
    _dense1 = tf.reshape(_pool_dr2
                         , [-1, _w['wd1'].get_shape().as_list()[0]])
    # FULLY CONNECTED LAYER 1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # FULLY CONNECTED LAYER 2
    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    # RETURN
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1
        , 'pool1_dr1': _pool_dr1, 'conv2': _conv2, 'pool2': _pool2
        , 'pool_dr2': _pool_dr2, 'dense1': _dense1, 'fc1': _fc1
        , 'fc_dr1': _fc_dr1, 'out': _out
    }
    return out
print ("NETWORK READY")

NETWORK READY


### DEFINE FUNCTIONS

In [12]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

# Functions! 
_pred = conv_basic(x, weights, biases, keepratio, use_gray)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=_pred))
#WEIGHT_DECAY_FACTOR = 0.0001
#l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
#cost = cost + WEIGHT_DECAY_FACTOR * l2_loss

optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
print ("FUNCTIONS READY")

FUNCTIONS READY


### OPTIMIZE

In [16]:
# Parameters
training_epochs = 400
batch_size      = 10
test_size       = 30
display_step    = 40

# Launch the graph
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [17]:
# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain / batch_size)+1
    # Loop over all batches
    for i in range(num_batch): 
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        feeds = {x: batch_xs, y: batch_ys, keepratio:0.7}
        sess.run(optm, feed_dict=feeds)
        # Compute average loss
        feeds = {x: batch_xs, y: batch_ys, keepratio:1.}
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys, keepratio:1.}
        train_acc = sess.run(accr, feed_dict=feeds)
        print (" Training accuracy: %.3f" % (train_acc))
        
        randidx = np.random.randint(ntest, size=test_size)
        test_batch_xs = testimg[randidx, :]
        test_batch_ys = testlabel[randidx, :]
        feeds = {x: test_batch_xs, y: test_batch_ys, keepratio:1.}
        #feeds = {x: testimg, y: testlabel, keepratio:1.}
        test_acc = sess.run(accr, feed_dict=feeds)
        print (" Test accuracy: %.3f" % (test_acc))
        
print ("Optimization Finished!")

Epoch: 000/400 cost: 4.205391235
 Training accuracy: 0.300
 Test accuracy: 0.167
Epoch: 040/400 cost: 0.010767017
 Training accuracy: 1.000
 Test accuracy: 0.433
Epoch: 080/400 cost: 0.009388100
 Training accuracy: 1.000
 Test accuracy: 0.300
Epoch: 120/400 cost: 0.004812382
 Training accuracy: 1.000
 Test accuracy: 0.333
Epoch: 160/400 cost: 0.008097463
 Training accuracy: 1.000
 Test accuracy: 0.233
Epoch: 200/400 cost: 0.004928074
 Training accuracy: 1.000
 Test accuracy: 0.433
Epoch: 240/400 cost: 0.000802461
 Training accuracy: 1.000
 Test accuracy: 0.267


KeyboardInterrupt: 

### CLOSE SESSION

In [15]:
sess.close()
print ("Session closed.")

Session closed.
